<a href="https://colab.research.google.com/github/hbparache/SAR_ARD_opensource_comparison/blob/main/RMSE_NP_short.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import math
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error 

In [ ]:
# Connecting to Google Drive
from google.colab import drive
drive.mount ('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
ls

drive/  sample_data/


In [ ]:
cd /content/drive/MyDrive/RMSE_prep/NP

/content/drive/MyDrive/RMSE_prep/NP


In [ ]:
ls

GAMMA_flat_vh.csv  GAMMA_steep_vh.csv  GTC_flat_vh.csv  GTC_steep_vh.csv
GAMMA_flat_vv.csv  GAMMA_steep_vv.csv  GTC_flat_vv.csv  GTC_steep_vv.csv


Read in data

In [ ]:
#truth = pd.read_csv("GAMMA_flat_vh.csv") # 'truth', always gamma
#truth = pd.read_csv("GAMMA_flat_vv.csv")
truth = pd.read_csv("GAMMA_steep_vh.csv")
#truth = pd.read_csv("GAMMA_steep_vv.csv")

#Swap the predicted values

#pred = pd.read_csv("GRD_flat_vh.csv") # 'predicted'
#pred = pd.read_csv("GRD_flat_vv.csv") 
#pred = pd.read_csv("GRD_steep_vh.csv")
#pred = pd.read_csv("GRD_steep_vv.csv")

#pred = pd.read_csv("GTC_flat_vh.csv") # 'predicted'
#pred = pd.read_csv("GTC_flat_vv.csv") 
pred = pd.read_csv("GTC_steep_vh.csv")
#pred = pd.read_csv("GTC_steep_vv.csv")

#pred = pd.read_csv("SNAP_flat_vh.csv") # 'predicted'
#pred = pd.read_csv("SNAP_flat_vv.csv") 
#pred = pd.read_csv("SNAP_steep_vh.csv")
#pred = pd.read_csv("SNAP_steep_vv.csv")

#pred = pd.read_csv("ISCE_flat_vh.csv") # 'predicted'
#pred = pd.read_csv("ISCE_flat_vv.csv") 
#pred = pd.read_csv("ISCE_steep_vh.csv")
#pred = pd.read_csv("ISCE_steep_vv.csv")

#get basic stats
print(truth['mean'].describe())
print(pred['mean'].describe())

#set "Date" column to be the date type in pandas, and put in a new column
truth['start_date'] = pd.to_datetime(truth['Date'])
truth['start_date'] = truth['start_date'].apply(lambda t: t.replace(second=0, minute=0, hour=0)) #remove seconds bc these won't match (Gamma & ISCE products won't have)
pred['start_date'] = pd.to_datetime(pred['Date'])
pred['start_date'] = pred['start_date'].apply(lambda t: t.replace(second=0, minute=0, hour=0)) #remove seconds bc these won't match (Gamma & ISCE products won't have)

#check for NaN values that may cause issues later.
print(truth['.geo'].isnull().values.any())
print(truth['start_date'].isnull().values.any())
print(pred['.geo'].isnull().values.any())
print(pred['start_date'].isnull().values.any())

#.geo and start_date columns tog should create a unique identifier for each data point (point location with mean from one day)
truth["new_index"] = truth["start_date"].map(str) + truth[".geo"].map(str)
pred["new_index"] = pred["start_date"].map(str) + pred[".geo"].map(str)

truth = truth.set_index("new_index")
pred = pred.set_index("new_index")

#combined dataframe
combo = truth.join(pred, lsuffix='_truth', rsuffix='_pred')
combo = combo[['mean_truth', 'mean_pred']]
print(combo) #235308 rows x 2 columns
combo.isnull().values.any() #yes null values
combo = combo.dropna()
print(combo.isnull().values.any()) #no null values

# Calculation of RMSE
RMSE = math.sqrt(mean_squared_error(combo['mean_truth'], combo['mean_pred'])) 
print ("Gamma to [product]: RMSE (pw) ", RMSE)

#Convert power to db: dB= 10* log (DN).
x = math.log(RMSE)
y = x*10
print("RMSE (dB)", y)

#pearsonr
print("Pearson R")
r, p = pearsonr(combo['mean_truth'], combo['mean_pred'])
print (r)
print (p)

count    203539.000000
mean          0.046663
std           0.177801
min         -21.325985
25%           0.022416
50%           0.037351
75%           0.058515
max          65.274452
Name: mean, dtype: float64
count     1.994060e+05
mean      1.181815e-01
std       2.168148e-01
min      7.306579e-305
25%       2.386229e-05
50%       5.416247e-03
75%       1.329139e-01
max       3.276133e+00
Name: mean, dtype: float64
False
False
False
False
                                                    mean_truth  mean_pred
new_index                                                                
2019-02-11 00:00:00{"type":"Point","coordinates...    0.039265   0.037538
2019-02-11 00:00:00{"type":"Point","coordinates...    0.010829   0.000004
2019-02-11 00:00:00{"type":"Point","coordinates...    0.078052   0.000764
2019-02-11 00:00:00{"type":"Point","coordinates...    0.039125   0.143289
2019-02-11 00:00:00{"type":"Point","coordinates...    0.032997   0.026434
...                                 